In [3]:
from fredapi import Fred
import pandas as pd

# 1. Connect to FRED
fred = Fred(api_key='e916710d165717e6348556cdce8111f3')

# 2. Download series (monthly, end‐of‐month)
series = {
    'CPI_YoY':  'CPIAUCNS',   # CPI all urban consumers (we’ll drop this later)
    'Unemployment': 'UNRATE', # Unemployment rate (%)
    'M2_YoY': 'M2SL',         # M2 money stock (will convert to YoY %)
    'FedFunds': 'FEDFUNDS',   # Effective federal funds rate
}

# Fetch each into a DataFrame
df_list = []
for name, code in series.items():
    s = fred.get_series(code)
    s = s.resample('M').last()           # ensure clean monthly index
    if name.endswith('_YoY'):
        # for CPI_YoY & M2_YoY: compute YoY % change
        s = s.pct_change(12) * 100
    df_list.append(s.rename(name))

# 3. Combine into one DataFrame
data = pd.concat(df_list, axis=1)

# ── Fix: drop the overlapping CPI_YoY column ────────────────────────────────
if 'CPI_YoY' in data.columns:
    data = data.drop(columns=['CPI_YoY'])

# 4. Load your CPI z‐score & regime labels
cpi = pd.read_csv(
    r'C:\Users\ghkjs\GitHub\regime-shift-classification\data\processed\cpi_labeled_regimes.csv',
    index_col=0,
    parse_dates=True
)

# 5. Merge all features with the target
df = data.join(cpi[['CPI_YoY', 'Zscore', 'Inflation_Regime']], how='inner')

# 6. Quick check
print(df.tail())

# 7. Save for modeling
df.to_csv(
    r'C:\Users\ghkjs\GitHub\regime-shift-classification\data\processed\inflation_regime_dataset.csv',
    float_format='%.4f'
)

C:\Users\ghkjs\AppData\Local\Temp\ipykernel_44612\203797381.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  s = s.resample('M').last()           # ensure clean monthly index


            Unemployment    M2_YoY  FedFunds   CPI_YoY    Zscore  \
2025-01-31           4.0  3.594553      4.33  3.000483 -0.487322   
2025-02-28           4.1  3.606216      4.33  2.821549 -0.563675   
2025-03-31           4.2  3.851417      4.33  2.390725 -0.746759   
2025-04-30           4.2  4.442374      4.33  2.311289 -0.805370   
2025-05-31           4.2  4.500643      4.33  2.354897 -0.818960   

            Inflation_Regime  
2025-01-31                 0  
2025-02-28                 0  
2025-03-31                 0  
2025-04-30                 0  
2025-05-31                 0  
